In [2]:
import torch
torch.cuda.set_device(5)

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

# POS Tag

## HMM

In [6]:
import nltk
from nltk.tag.hmm import HiddenMarkovModelTrainer

In [3]:
def open_file_pos_tag(path):
    with open(path, 'r',  encoding= 'unicode_escape') as f:
        train_data = []
        sentence = []
        for line in f:
            if line.strip() == "":
                if len(sentence) > 0:
                    train_data.append(sentence)
                    sentence = []
            else:
                word, pos = line.strip().split("\t")
                sentence.append((word, pos))
        if len(sentence) > 0:
            train_data.append(sentence)
    return train_data

In [6]:
train_data = open_file_pos_tag("datasets/idn-tagged-corpus-CSUI/IndoNLU-split/train_preprocess.conll")
val_data = open_file_pos_tag("datasets/idn-tagged-corpus-CSUI/IndoNLU-split/valid_preprocess.conll")
test_data = open_file_pos_tag("datasets/idn-tagged-corpus-CSUI/IndoNLU-split/test_preprocess.conll")

In [4]:
formatted_train_data = []
for sentence in train_data:
    tokens, tags = zip(*sentence)
    formatted_train_data.append(list(zip(tokens, tags)))

# Inisialisasi trainer HMM
trainer = HiddenMarkovModelTrainer()

# Latih model HMM dengan data pelatihan
tagger = trainer.train_supervised(formatted_train_data)

# Contoh penggunaan tagger
sentence = ["I", "love", "natural", "language", "processing"]
tags = tagger.tag(sentence)
print(tags)

NameError: name 'train_data' is not defined

In [19]:
predicted_tags = [tagger.tag([word[0] for word in sentence]) for sentence in test_data]

In [22]:
predicted_tags = [[word[1] for word in sentence] for sentence in predicted_tags]

In [24]:
true_labels = [[word[1] for word in sentence] for sentence in test_data]

In [7]:
# Train model HMM dengan menggunakan data train
trainer = HiddenMarkovModelTrainer()
model = trainer.train_supervised(train_data)

# Evaluasi performa model pada data validation
print("Accuracy on validation data:", model.accuracy(val_data))

# Lakukan tagging pada data test
predicted_tags = [model.tag(sentence) for sentence in test_data]

# Evaluasi performa model pada data test
print("Accuracy on test data:", model.accuracy(test_data))

Accuracy on validation data: 0.5480754366980987
Accuracy on test data: 0.5332609184208555


In [12]:
def remove_bio_tags(tags):
    # Menghapus tag BIO dari POS tag
    return [tag[2:] for tag in tags]



In [ ]:
y_true = [remove_bio_tags(tags) for tags in true_labels]
y_pred = [remove_bio_tags(tags) for tags in predicted_tags]

In [61]:
len(train_data)

8000

In [60]:
# print(classification_report(y_true, y_pred, zero_division=0))
print(flat_classification_report(y_true, y_pred, zero_division=0, labels=None))

              precision    recall  f1-score   support

          CC       0.98      0.40      0.57       808
          CD       0.99      0.43      0.60      1038
          DT       1.00      0.75      0.86        84
          FW       0.33      0.14      0.20        14
          IN       0.96      0.40      0.56      2213
          JJ       0.87      0.40      0.55       856
          MD       0.97      0.53      0.68       443
         NEG       1.00      0.52      0.68       202
          NN       0.35      0.98      0.52      6596
         NND       0.94      0.48      0.63       199
         NNP       0.93      0.35      0.50      3973
          OD       0.68      0.29      0.41        58
          PR       1.00      0.49      0.66       676
         PRP       0.98      0.47      0.64      1196
          RB       0.94      0.51      0.66       445
          RP       0.82      0.50      0.62        18
          SC       0.95      0.42      0.59      1349
         SYM       0.00    

In [38]:
from seqeval.metrics import classification_report
from  sklearn_crfsuite.metrics import flat_classification_report 

print(classification_report(true_labels, predicted_tags, zero_division=0))
print(flat_classification_report(true_labels, predicted_tags, zero_division=0, labels=None))

              precision    recall  f1-score   support

          CC       0.98      0.40      0.57       808
          CD       0.98      0.43      0.60      1005
          DT       1.00      0.75      0.86        84
          FW       0.33      0.14      0.20        14
          IN       0.96      0.40      0.56      2212
          JJ       0.88      0.40      0.55       847
          MD       0.97      0.53      0.68       443
         NEG       1.00      0.52      0.68       202
          NN       0.32      0.92      0.47      6207
         NND       0.94      0.48      0.64       198
         NNP       0.92      0.35      0.50      3921
          OD       0.68      0.30      0.42        56
          PR       0.98      0.48      0.65       660
         PRP       0.98      0.50      0.66       814
          RB       0.93      0.50      0.65       432
          RP       0.82      0.50      0.62        18
          SC       0.95      0.42      0.59      1331
         SYM       0.00    

## CRF

In [83]:
import nltk
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report

In [84]:
def word2features(sent, i):
    word = sent[i][0]
    features = {
        'word': word,
        'bias': 1.0,
    }
    if i > 0:
        prev_word = sent[i-1][0]
        features.update({
            'prev_word': prev_word,
            'prev_word.lower()': prev_word.lower(),
            'prev_word.isupper()': prev_word.isupper(),
            'prev_word.istitle()': prev_word.istitle(),
            'prev_word.isdigit()': prev_word.isdigit(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        next_word = sent[i+1][0]
        features.update({
            'next_word': next_word,
            'next_word.lower()': next_word.lower(),
            'next_word.isupper()': next_word.isupper(),
            'next_word.istitle()': next_word.istitle(),
            'next_word.isdigit()': next_word.isdigit(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [85]:
# Convert input data to feature set
X_train = [sent2features(sent) for sent in train_data]
y_train = [sent2labels(sent) for sent in train_data]
X_val = [sent2features(sent) for sent in val_data]
y_val = [sent2labels(sent) for sent in val_data]
X_test = [sent2features(sent) for sent in test_data]
y_test = [sent2labels(sent) for sent in test_data]

In [86]:
# Inisialisasi model CRF dan trainer
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=50,
    all_possible_transitions=True
)

# Latih model dengan data pelatihan
crf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = crf.predict(X_test)

In [ ]:
y_test = [remove_bio_tags(tags) for tags in y_test]
y_pred = [remove_bio_tags(tags) for tags in y_pred]

In [14]:
# Evaluasi performa model dengan mencetak laporan klasifikasi
print(metrics.flat_classification_report(y_test, y_pred, digits=4))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

          CC     0.9877    0.9950    0.9914       808
          CD     0.9674    0.9143    0.9401      1038
          DT     0.9882    1.0000    0.9941        84
          FW     0.0909    0.0714    0.0800        14
          IN     0.9620    0.9616    0.9618      2213
          JJ     0.9032    0.8505    0.8761       856
          MD     0.9865    0.9932    0.9899       443
         NEG     1.0000    0.9752    0.9875       202
          NN     0.8855    0.9500    0.9166      6596
         NND     0.9882    0.8392    0.9076       199
         NNP     0.9106    0.8789    0.8945      3973
          OD     0.9048    0.6552    0.7600        58
          PR     0.9955    0.9911    0.9933       676
         PRP     0.9983    0.9941    0.9962      1196
          RB     0.9720    0.9371    0.9542       445
          RP     0.9412    0.8889    0.9143        18
          SC     0.9348    0.9466    0.9407      1349
         SYM     1.0000    

In [8]:
# Prediksi label untuk data pengujian
y_pred = crf.predict(X_test)

# Evaluasi performa model dengan mencetak laporan klasifikasi
print(metrics.flat_classification_report(y_test, y_pred))

y_pred_test = crf.predict(X_val)
print(metrics.flat_classification_report(y_test, y_pred))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        B-CC       0.99      1.00      0.99       808
        B-CD       0.97      0.91      0.94      1005
        B-DT       0.99      1.00      0.99        84
        B-FW       0.09      0.07      0.08        14
        B-IN       0.96      0.97      0.97      2212
        B-JJ       0.91      0.85      0.88       847
        B-MD       0.99      0.99      0.99       443
       B-NEG       0.99      0.98      0.98       202
        B-NN       0.87      0.95      0.91      6207
       B-NND       0.98      0.84      0.90       198
       B-NNP       0.92      0.88      0.90      3921
        B-OD       0.91      0.70      0.79        56
        B-PR       1.00      0.99      0.99       660
       B-PRP       1.00      0.99      1.00       814
        B-RB       0.97      0.94      0.96       432
        B-RP       0.94      0.89      0.91        18
        B-SC       0.94      0.95      0.95      1331
       B-SYM       1.00    

## BERT

In [71]:
import os
import torch
# os.environ['CUDA_VISIBLE_DEVICES'] = '5'
os.environ["TOKENIZERS_PARALLELISM"] = "false"
torch.cuda.set_device(5) 

In [15]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-large-p2")
model = AutoModelForTokenClassification.from_pretrained("indobenchmark/indobert-large-p2")


In [35]:
model_name = "indobenchmark/indobert-large-p2"

In [36]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [22]:
train_sentences = [[word[0] for word in sentence] for sentence in train_data]
val_sentences = [[word[0] for word in sentence] for sentence in val_data]
test_sentences = [[word[0] for word in sentence] for sentence in test_data]

In [23]:
train_labels = [[s[1] for s in sentence]for sentence in train_data]
val_labels = [[s[1] for s in sentence]for sentence in val_data]
test_labels = [[s[1] for s in sentence]for sentence in test_data]

In [24]:
unique_labels = list(set([lab for label in train_labels + val_labels + test_labels for lab in label]))
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

In [16]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [17]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [18]:
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [26]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [20]:
class dataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.len = len(sentences)
        # self.data = dataframe
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.sentences[index]
        word_labels = self.labels[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                                  is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] != 0:
                # overwrite label
                encoded_labels[idx] = labels[i]
                i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

    def __len__(self):
        return self.len

In [27]:
training_set = dataset(train_sentences, train_labels, tokenizer, MAX_LEN)
validation_set = dataset(val_sentences, val_labels, tokenizer, MAX_LEN)
testing_set = dataset(test_sentences, test_labels, tokenizer, MAX_LEN)

In [28]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(validation_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [71]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_labels), id2label=ids_to_labels, label2id=labels_to_ids)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [72]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [111]:
from tqdm import tqdm

def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    val_loss, val_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    nb_val_examples, nb_val_steps = 0, 0
    tr_preds, tr_labels = [], []
    val_preds, val_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader)):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = tr_loss/nb_tr_steps
        #     print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_step = tr_loss/nb_tr_steps
    
    model.eval()
    for idx, batch in enumerate(val_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        val_logits = outputs[1]
        val_loss += loss.item()

        nb_val_steps += 1
        nb_val_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = val_loss/nb_val_steps
        #     print(f"Validation loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = val_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        val_labels.extend(labels)
        val_preds.extend(predictions)

        tmp_val_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        val_accuracy += tmp_val_accuracy
    

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Validation loss epoch: {val_loss/nb_val_steps}")
    print(f"Validation accuracy epoch: {val_accuracy/nb_val_steps}")

In [74]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1


100%|███████████████████████████████████████████████████████████████████████████████| 2000/2000 [10:37<00:00,  3.14it/s]


Training loss epoch: 0.27984153365856035
Training accuracy epoch: 0.9266460713881918
Validation loss epoch: 0.12601732651889325
Validation accuracy epoch: 0.9624888496874521
Training epoch: 2


100%|███████████████████████████████████████████████████████████████████████████████| 2000/2000 [10:36<00:00,  3.14it/s]


Training loss epoch: 0.08664289848715999
Training accuracy epoch: 0.9731134686967032
Validation loss epoch: 0.11154031656309962
Validation accuracy epoch: 0.9660649282009256
Training epoch: 3


...

Training loss epoch: 0.05764477203600109
Training accuracy epoch: 0.9811682522412064
Validation loss epoch: 0.11902348667476326
Validation accuracy epoch: 0.9659174528564203
Training epoch: 4


100%|███████████████████████████████████████████████████████████████████████████████| 2000/2000 [10:37<00:00,  3.14it/s]


Training loss epoch: 0.041628227129229346
Training accuracy epoch: 0.9861309213971499
Validation loss epoch: 0.12708195730391889
Validation accuracy epoch: 0.9677231101872287
Training epoch: 5


100%|███████████████████████████████████████████████████████████████████████████████| 2000/2000 [10:37<00:00,  3.14it/s]


Training loss epoch: 0.029889684904555906
Training accuracy epoch: 0.9896490759087789
Validation loss epoch: 0.12473972872737796
Validation accuracy epoch: 0.9684536755737387


In [109]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [29]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    # with torch.no_grad():
    for idx, batch in enumerate(testing_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        eval_logits = outputs[1]

        eval_loss += loss.item()

        nb_eval_steps += 1
        nb_eval_examples += labels.size(0)

        # if idx % 100==0:
        #     loss_step = eval_loss/nb_eval_steps
        #     print(f"Validation loss per 100 evaluation steps: {loss_step}")

        # compute evaluation accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        eval_labels.extend(labels)
        eval_preds.extend(predictions)

        tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        eval_accuracy += tmp_eval_accuracy

    labels = [[model.config.id2label[id.item()]] for id in eval_labels]
    predictions = [[model.config.id2label[id.item()]] for id in eval_preds]
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [114]:
device = "cuda"

In [30]:
labels, predictions = valid(model, testing_loader)

Validation Loss: 0.13980565264965725
Validation Accuracy: 0.9662581415474747


In [31]:
y_test = [remove_bio_tags(tags) for tags in labels]
y_pred = [remove_bio_tags(tags) for tags in predictions]

In [35]:
print(metrics.flat_classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

          CC     0.9938    0.9975    0.9957       808
          CD     0.9912    0.9769    0.9840      1038
          DT     1.0000    1.0000    1.0000        84
          FW     0.5000    0.2857    0.3636        14
          IN     0.9740    0.9634    0.9687      2213
          JJ     0.9353    0.9451    0.9402       856
          MD     0.9865    0.9887    0.9876       443
         NEG     1.0000    0.9851    0.9925       202
          NN     0.9624    0.9659    0.9641      6596
         NND     0.9492    0.9397    0.9444       199
         NNP     0.9497    0.9514    0.9506      3973
          OD     0.9259    0.8621    0.8929        58
          PR     0.9956    0.9956    0.9956       676
         PRP     1.0000    1.0000    1.0000      1196
          RB     0.9579    0.9708    0.9643       445
          RP     1.0000    0.7222    0.8387        18
          SC     0.9413    0.9622    0.9516      1349
         SYM     1.0000    

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [10]:
from seqeval.metrics import classification_report
from  sklearn_crfsuite.metrics import flat_classification_report 

# print(classification_report(labels, predictions))
# print(flat_classification_report(labels, predictions))

In [95]:
sentence = "BJ Habibie adalah Presiden Indonesia ke-3"

In [96]:
import torch
def predict(model, tokenizer, sentence):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentence.split(),
                    is_split_into_words = True,
                    return_offsets_mapping=True, 
                    return_tensors="pt")
    
    model.to(device)
    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [model.config.id2label[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
        #only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
        else:
            continue

    return sentence.split(), prediction

In [97]:
predict(model, tokenizer, sentence)

(['BJ', 'Habibie', 'adalah', 'Presiden', 'Indonesia', 'ke-3'],
 ['B-NNP', 'B-NNP', 'B-VB', 'B-NNP', 'B-NNP', 'B-OD'])

In [99]:
inputs = tokenizer(sentence.split(),
                    is_split_into_words = True,
                    return_offsets_mapping=True, 
                    return_tensors="pt")

In [111]:
tokenizer.decode(inputs["input_ids"].squeeze().tolist())

'[CLS] bj habibie adalah presiden indonesia ke - 3 [SEP]'

In [110]:
inputs["input_ids"].squeeze().tolist()

[2, 18480, 16539, 154, 1871, 300, 43, 30469, 283, 3]

In [101]:
tokenizer.decode(inputs["input_id"])

tensor([[[0, 0],
         [0, 2],
         [0, 7],
         [0, 6],
         [0, 8],
         [0, 9],
         [0, 2],
         [2, 3],
         [3, 4],
         [0, 0]]])

In [60]:
import torch
print(torch.cuda.current_device())

5


In [113]:
import os

directory = "./indobenchmark/indobert-large-p2"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

All files saved


## Bi-LSTM

In [36]:
all_data = train_data + val_data + test_data

In [38]:
maxlen = max([len(s) for s in all_data])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 84


In [39]:
words = list(set([word[0] for sentence in all_data for word in sentence]))
words.append("ENDPAD")

In [40]:
n_words = len(words); n_words

17586

In [41]:
tags = list(set([word[1] for sentence in all_data for word in sentence]))

In [42]:
n_tags = len(tags); n_tags

41

In [43]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [45]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = [[word2idx[w[0]] for w in s] for s in train_data]

In [46]:
X_test = [[word2idx[w[0]] for w in s] for s in test_data]

In [47]:
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=n_words - 1)

In [48]:
X_test = pad_sequences(maxlen=maxlen, sequences=X_test, padding="post",value=n_words - 1)

In [49]:
y = [[tag2idx[w[1]] for w in s] for s in train_data]

In [50]:
y_test = [[tag2idx[w[1]] for w in s] for s in test_data]

In [54]:
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag2idx["B-Z"])

In [55]:
y_test = pad_sequences(maxlen=maxlen, sequences=y_test, padding="post", value=tag2idx["B-Z"])

In [56]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [57]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import tensorflow as tf

In [58]:
input = Input(shape=(maxlen,))
model = Embedding(input_dim=n_words, output_dim=maxlen, input_length=maxlen)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer

In [59]:
model = Model(input, out)

In [60]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [61]:
import numpy as np

history = model.fit(X, np.array(y), batch_size=32, epochs=5, validation_split=0.05, verbose=1)

Epoch 1/5
238/238 [==============================] - 100s 395ms/step - loss: 0.9254 - precision: 0.9861 - recall: 0.6564 - val_loss: 0.6556 - val_precision: 0.9988 - val_recall: 0.7134
Epoch 2/5
238/238 [==============================] - 81s 342ms/step - loss: 0.3521 - precision: 0.9920 - recall: 0.8380 - val_loss: 0.1899 - val_precision: 0.9848 - val_recall: 0.9303
Epoch 3/5
238/238 [==============================] - 78s 328ms/step - loss: 0.0960 - precision: 0.9889 - recall: 0.9688 - val_loss: 0.1177 - val_precision: 0.9851 - val_recall: 0.9566
Epoch 4/5
238/238 [==============================] - 78s 326ms/step - loss: 0.0558 - precision: 0.9901 - recall: 0.9823 - val_loss: 0.0989 - val_precision: 0.9825 - val_recall: 0.9641
Epoch 5/5
238/238 [==============================] - 76s 320ms/step - loss: 0.0411 - precision: 0.9913 - recall: 0.9868 - val_loss: 0.0939 - val_precision: 0.9818 - val_recall: 0.9667


In [62]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out

def tolabel(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out

In [63]:
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = tolabel(y_test)

33/33 [==============================] - 2s 41ms/step


In [142]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [143]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 0.0%


In [64]:
print(classification_report(test_labels, pred_labels))

ValueError: multiclass-multioutput is not supported

In [67]:
y_test = [remove_bio_tags(tags) for tags in test_labels]
y_pred = [remove_bio_tags(tags) for tags in pred_labels]

In [80]:
y_pred_2 = [j[:len(i)] for i, j in zip(test_data, y_pred)]

In [81]:
y_test_2 = [j[:len(i)] for i, j in zip(test_data, y_test)]

In [74]:
len(test_data[0])

30

In [82]:
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=y_pred_2, y_true=y_test_2, digits=4)
print(report)

              precision    recall  f1-score   support

          CC     0.9817    0.9950    0.9883       808
          CD     0.9751    0.9046    0.9385      1038
          DT     1.0000    1.0000    1.0000        84
          FW     0.1351    0.3571    0.1961        14
          IN     0.9397    0.9652    0.9523      2213
          JJ     0.8737    0.8645    0.8691       856
          MD     0.9775    0.9819    0.9797       443
         NEG     0.9949    0.9752    0.9850       202
          NN     0.8449    0.9547    0.8964      6596
         NND     0.9486    0.8342    0.8877       199
         NNP     0.9251    0.7891    0.8517      3973
          OD     0.7111    0.5517    0.6214        58
          PR     0.9908    0.9586    0.9744       676
         PRP     0.9909    0.9983    0.9946      1196
          RB     0.9564    0.9371    0.9467       445
          RP     0.6957    0.8889    0.7805        18
          SC     0.9302    0.9481    0.9391      1349
         SYM     0.5000    

# NER

In [5]:
train_data_ner = open_file_pos_tag("indonlu/dataset/nergrit_ner-grit/train_preprocess.txt")
val_data_ner = open_file_pos_tag("indonlu/dataset/nergrit_ner-grit/valid_preprocess.txt")
test_data_ner = open_file_pos_tag("indonlu/dataset/nergrit_ner-grit/test_preprocess.txt")

## CRF

In [87]:
def word2features_ner(df, i):
    word = df[i][0]
    features = {
        "bias": 1.0,
        "word.lower()": word.lower(),
        "word.isupper()": word.isupper(),
        "word.istitle()": word.istitle(),
        "word.isdigit()": word.isdigit(),
    }
    if i > 0:
        prev_word = df[i-1][0]
        features.update({
            "-1:word.lower()": prev_word.lower(),
            "-1:word.istitle()": prev_word.istitle(),
            "-1:word.isupper()": prev_word.isupper(),
        })
    else:
        features["BOS"] = True
    if i < len(df)-1:
        next_word = df[i+1][0]
        features.update({
            "+1:word.lower()": next_word.lower(),
            "+1:word.istitle()": next_word.istitle(),
            "+1:word.isupper()": next_word.isupper(),
        })
    else:
        features["EOS"] = True
    return features

def sent2features_ner(sent):
    return [word2features_ner(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [90]:
# Convert input data to feature set
X_train_ner = [sent2features_ner(sent) for sent in train_data_ner]
y_train_ner = [sent2labels(sent) for sent in train_data_ner]
X_val_ner = [sent2features_ner(sent) for sent in val_data_ner]
y_val_ner = [sent2labels(sent) for sent in val_data_ner]
X_test_ner = [sent2features_ner(sent) for sent in test_data_ner]
y_test_ner = [sent2labels(sent) for sent in test_data_ner]

In [91]:
crf_ner = sklearn_crfsuite.CRF(algorithm="lbfgs", c1=0.1, c2=0.1, max_iterations=100)
crf_ner.fit(X_train_ner, y_train_ner)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', c1=0.1, c2=0.1, keep_tempfiles=None, max_iterations=100)

In [92]:
y_pred = crf_ner.predict(X_val_ner)
print(metrics.flat_classification_report(y_val_ner, y_pred))

y_pred_2 = crf_ner.predict(X_test_ner)
print(metrics.flat_classification_report(y_test_ner, y_pred_2))

                precision    recall  f1-score   support

B-ORGANISATION       0.68      0.45      0.54       121
      B-PERSON       0.71      0.59      0.64       213
       B-PLACE       0.84      0.73      0.78       328
I-ORGANISATION       0.55      0.49      0.52       114
      I-PERSON       0.77      0.71      0.74       209
       I-PLACE       0.85      0.76      0.80       277
             O       0.95      0.98      0.96      5721

      accuracy                           0.92      6983
     macro avg       0.76      0.67      0.71      6983
  weighted avg       0.92      0.92      0.92      6983

                precision    recall  f1-score   support

B-ORGANISATION       0.64      0.40      0.49       124
      B-PERSON       0.66      0.64      0.65       207
       B-PLACE       0.81      0.72      0.76       357
I-ORGANISATION       0.61      0.53      0.57       150
      I-PERSON       0.68      0.70      0.69       161
       I-PLACE       0.75      0.76      0.7

In [93]:
y_pred_2 = crf_ner.predict(X_test_ner)
print(metrics.flat_classification_report(y_test_ner, y_pred_2, digits=4))

                precision    recall  f1-score   support

B-ORGANISATION     0.6447    0.3952    0.4900       124
      B-PERSON     0.6600    0.6377    0.6486       207
       B-PLACE     0.8082    0.7199    0.7615       357
I-ORGANISATION     0.6061    0.5333    0.5674       150
      I-PERSON     0.6829    0.6957    0.6892       161
       I-PLACE     0.7483    0.7643    0.7562       280
             O     0.9593    0.9770    0.9681      5575

      accuracy                         0.9179      6854
     macro avg     0.7299    0.6747    0.6973      6854
  weighted avg     0.9139    0.9179    0.9151      6854



In [94]:
from seqeval.metrics import classification_report

y_pred_2 = crf_ner.predict(X_test_ner)
print(classification_report(y_test_ner, y_pred_2, digits=4))

              precision    recall  f1-score   support

ORGANISATION     0.5789    0.3548    0.4400       124
      PERSON     0.6450    0.6232    0.6339       207
       PLACE     0.7862    0.7003    0.7407       357

   micro avg     0.7121    0.6148    0.6599       688
   macro avg     0.6700    0.5594    0.6049       688
weighted avg     0.7063    0.6148    0.6544       688



In [20]:
test_data_ner[0]

[('Joetata', 'B-PERSON'),
 ('Hadihardaja', 'I-PERSON'),
 ('dan', 'O'),
 ('dihadiri', 'O'),
 ('oleh', 'O'),
 ('Rektor', 'O'),
 ('Undip', 'B-ORGANISATION'),
 ('Prof', 'O'),
 ('.', 'O')]

In [30]:
y_12 = [['B-PERSON',
  'I-PERSON',
    "B-ORGANISATION",
  'O',
  'O',
  'O',
  'B-ORGANISATION',
  'I-ORGANISATION',
  'I-ORGANISATION',
  'B-ORGANISATION', "B-PERSON"]]
y_123 = [['B-PERSON', 'I-PERSON', 'B-PERSON','O', 'O', 'O', 'O', 'B-ORGANISATION', 'O', 'B-ORGANISATION', "O"]]

print(classification_report(y_123, y_12))

              precision    recall  f1-score   support

ORGANISATION       0.33      0.50      0.40         2
      PERSON       0.50      0.50      0.50         2

   micro avg       0.40      0.50      0.44         4
   macro avg       0.42      0.50      0.45         4
weighted avg       0.42      0.50      0.45         4



## BERT

In [3]:
# mengubah dataset menjadi format yang dapat digunakan oleh model BERT
sentences = []
labels = []
for sentence in train_data_ner:
    current_sentence = []
    current_labels = []
    for word, label in sentence:
        current_sentence.append(word)
        current_labels.append(label)
    sentences.append(" ".join(current_sentence))
    labels.append(current_labels)

In [17]:
# mengubah label menjadi label ID
label_list = list(set([label for labels in labels for label in labels]))
label_map = {label: i for i, label in enumerate(label_list)}
labels = [[label_map[label] for label in sentence_labels] for sentence_labels in labels]

In [22]:
id_to_label = {v: k for k,v in label_map.items()}

In [24]:
# mengubah kata dan label menjadi token dan label ID
tokenized_inputs = tokenizer(sentences, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
# labels = [torch.tensor(l, device=device) for l in labels]

In [92]:
# decoded_text = tokenizer.decode(tokenized_inputs['input_ids'][0], skip_special_tokens=True)

In [45]:
# print(tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0]))

In [46]:
# tokenized_inputs.word_ids()

In [48]:
def align_label(tokenized_input, labels):
    word_ids = tokenized_input.word_ids()
    prev_word_idx = None
    label_ids = []
    
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
            
        elif word_idx != prev_word_idx:
            try:
                label_ids.append(label_map[labels[word_idx]])
            except:
                label_ids.append(-100)
                
        else:
            labels_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
        prev_word_idx = word_idx
        
    return labels_ids   

In [52]:
label = labels[4]

In [71]:
type(tokenized_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [70]:
tokenized_inputs[4].word_ids()

TypeError: 'list' object is not callable

In [55]:
label_all_tokens = True
new_label = align_label(tokenized_inputs[4], label)

TypeError: 'list' object is not callable

In [105]:
labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True)
tokenized_inputs = {k: v.to(device) for k, v in tokenized_inputs.items()}

In [109]:
tokenized_inputs['input_ids'][0]

tensor([    2,  7733,    57,   618,  1934,  2002,   351,  5597,   271,  3848,
         1087,  2240,  4554,  6240, 16330, 30468,   501,   662,  7020, 20131,
        30473,   662, 29921, 12635,  1154,  5671, 26478, 30473,    41, 26212,
         5429,   947,  8146,  1633,  8023, 30470,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [108]:
labels[0]

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

In [ ]:
## Lanjutkan eksplor, coreference resolution (kode kak valentina, kak irwanto), baca blog, dependency, constituency parsing (xlm-roberta-base), 
## POS and NER (IndoNLU)
## Rulenya hilangin angka 

## BERT 1

In [74]:
# https://towardsdatascience.com/named-entity-recognition-with-bert-in-pytorch-a454405e0b6a

In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [213]:
def data_to_sentence_label(data):
    sentences = []
    labels = []
    for sentence in data:
        current_sentence = []
        current_labels = []
        for word, label in sentence:
            current_sentence.append(word)
            current_labels.append(label)
        sentences.append(" ".join(current_sentence))
        labels.append(current_labels)
    return (sentences, labels)

In [214]:
unique_labels = list(set([label for labels in labels for label in labels]))

In [215]:
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

In [216]:
from transformers import AutoTokenizer

model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [217]:
import torch

def align_label(texts, labels, is_label_all_tokens):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if is_label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, sentences, labels):
        self.texts = [tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for sentence in sentences]
        self.labels = [align_label(sentence, label, False) for sentence, label in zip(sentences, labels)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [218]:
from transformers import BertForTokenClassification

class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

In [219]:
from torch.utils.data import DataLoader
from torch.optim import SGD
from tqdm import tqdm

def train_loop(model, df_train, df_val):
    train = data_to_sentence_label(df_train)
    val = data_to_sentence_label(df_val)
    
    train_dataset = DataSequence(train[0], train[1])
    val_dataset = DataSequence(val[0], val[1])

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):
                logits_clean = logits[i][train_label[i] != -100]
                label_clean = train_label[i][train_label[i] != -100]
                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_train += acc
                total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):
                logits_clean = logits[i][val_label[i] != -100]
                label_clean = val_label[i][val_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_val += acc
                total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

In [220]:
LEARNING_RATE = 5e-3
EPOCHS = 5
BATCH_SIZE = 2

model = BertModel()
train_loop(model, train_data_ner, val_data_ner)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|                                                                                           | 1/836 [00:00<08:13,  1.69it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████████████████████████████████████████████████████| 836/836 [00:44<00:00, 19.29it/s]

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|████████████████████████████████████████████████████████████████████████████████████████▉| 835/836 [00:44<00:00, 19.17it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████████████████████████████████████████████████████| 836/836 [00:44<00:00, 18.75it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
def align_word_ids(texts, is_label_all_tokens):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if is_label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx
    return label_ids


def evaluate_one_text(model, sentence):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence, False)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)

In [ ]:
sentence = 'Bank BNI miliknya Bambang Pacul'

In [ ]:
text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

mask = text['attention_mask'].to(device)
input_id = text['input_ids'].to(device)
label_ids = torch.Tensor(align_word_ids(sentence, False)).unsqueeze(0).to(device)

In [ ]:
label_ids

In [ ]:
logits = model(input_id, mask, None)
logits_clean = logits[0][label_ids != -100]

In [ ]:
predictions = logits_clean.argmax(dim=1).tolist()

In [ ]:
logits_clean

In [ ]:
text

In [225]:
evaluate_one_text(model, 'Bank Indonesia miliknya Bambang Pacul')

Bank Indonesia miliknya Bambang Pacul
['B-ORGANISATION', 'I-ORGANISATION', 'O', 'B-PERSON', 'I-PERSON']


In [289]:
t = torch.tensor([[[[[1,2,3], [1,2,3]]]]])

In [290]:
t.shape

torch.Size([1, 1, 1, 2, 3])

In [ ]:
t.squeeze(1).shape

## BERT 2

In [18]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [76]:
model_name = "indobenchmark/indobert-large-p2"

In [77]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [149]:
train_sentences = [[word[0] for word in sentence] for sentence in train_data_ner]
val_sentences = [[word[0] for word in sentence] for sentence in val_data_ner]
test_sentences = [[word[0] for word in sentence] for sentence in test_data_ner]

In [150]:
train_labels = [[s[1] for s in sentence]for sentence in train_data_ner]
val_labels = [[s[1] for s in sentence]for sentence in val_data_ner]
test_labels = [[s[1] for s in sentence]for sentence in test_data_ner]

In [59]:
unique_labels = list(set([lab for label in labels for lab in label]))
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

In [28]:
tokenized_input = tokenizer(sentences[0], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
word_ids = tokenized_input.word_ids()

In [64]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [213]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [211]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [87]:
tokenizer

BertTokenizerFast(name_or_path='indobenchmark/indobert-large-p2', vocab_size=30521, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [214]:
class dataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.len = len(sentences)
        # self.data = dataframe
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.sentences[index]
        word_labels = self.labels[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                                  is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] != 0:
                # overwrite label
                encoded_labels[idx] = labels[i]
                i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

    def __len__(self):
        return self.len

In [215]:
training_set = dataset(train_sentences, train_labels, tokenizer, MAX_LEN)
validation_set = dataset(val_sentences, val_labels, tokenizer, MAX_LEN)
testing_set = dataset(test_sentences, test_labels, tokenizer, MAX_LEN)

In [91]:
training_set[0]

{'input_ids': tensor([    2,  7733,    57,   618,  1934,  2002,   351,  5597,   271,  3848,
          1087,  2240,  4554,  6240, 16330, 30468,   501,   662,  7020, 20131,
         30473,   662, 29921, 12635,  1154,  5671, 26478, 30473,    41, 26212,
          5429,   947,  8146,  1633,  8023, 30470,     3,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [95]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
    print('{0:10}  {1}'.format(token, label))

[CLS]       -100
kontribusi  6
##nya       -100
terhadap    6
industri    6
musik       6
telah       6
mengumpulkan  6
banyak      6
prestasi    6
termasuk    6
lima        6
gram        6
##my        -100
awards      6
,           6
serta       6
dua         6
belas       6
nominasi    6
;           6
dua         6
gui         6
##nn        -100
##ess       -100
world       6
records     6
;           6
dan         6
penjualannya  6
diperkirakan  6
sekitar     6
64          6
juta        6
rekaman     6
.           6
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100

In [216]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(validation_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [217]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_labels))
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [218]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(2.2864, device='cuda:0', grad_fn=<NllLossBackward0>)

In [219]:
print(outputs[0])

tensor(2.2864, device='cuda:0', grad_fn=<NllLossBackward0>)


In [220]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 7])

In [221]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [233]:
from tqdm import tqdm

In [234]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    val_loss, val_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    nb_val_examples, nb_val_steps = 0, 0
    tr_preds, tr_labels = [], []
    val_preds, val_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader)):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = tr_loss/nb_tr_steps
        #     print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_step = tr_loss/nb_tr_steps
    
    model.eval()
    for idx, batch in enumerate(val_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        val_logits = outputs[1]
        val_loss += loss.item()

        nb_val_steps += 1
        nb_val_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = val_loss/nb_val_steps
        #     print(f"Validation loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = val_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        val_labels.extend(labels)
        val_preds.extend(predictions)

        tmp_val_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        val_accuracy += tmp_val_accuracy
    

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Validation loss epoch: {val_loss/nb_val_steps}")
    print(f"Validation accuracy epoch: {val_accuracy/nb_val_steps}")

In [235]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1


100%|█████████████████████████████████████████████████████████████████████████████████████████| 418/418 [02:13<00:00,  3.14it/s]


Training loss epoch: 0.19877772072428151
Training accuracy epoch: 0.936759493316805
Validation loss epoch: 0.13621767146407432
Validation accuracy epoch: 0.9606129314763331
Training epoch: 2


100%|█████████████████████████████████████████████████████████████████████████████████████████| 418/418 [02:13<00:00,  3.13it/s]


Training loss epoch: 0.10228279978259686
Training accuracy epoch: 0.9677660975933049
Validation loss epoch: 0.12778205547552063
Validation accuracy epoch: 0.9599998955091337
Training epoch: 3


...

Training loss epoch: 0.0671822764285791
Training accuracy epoch: 0.9795311489403662
Validation loss epoch: 0.15111698213753835
Validation accuracy epoch: 0.9607256471815573
Training epoch: 4


100%|█████████████████████████████████████████████████████████████████████████████████████████| 418/418 [02:13<00:00,  3.13it/s]


Training loss epoch: 0.04620116308435094
Training accuracy epoch: 0.9861182150480817
Validation loss epoch: 0.16005088876765924
Validation accuracy epoch: 0.9625524508701846
Training epoch: 5


100%|█████████████████████████████████████████████████████████████████████████████████████████| 418/418 [02:13<00:00,  3.14it/s]


Training loss epoch: 0.033476605006960404
Training accuracy epoch: 0.9900565997406116
Validation loss epoch: 0.1716710735586876
Validation accuracy epoch: 0.9614848438312718


In [236]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [[ids_to_labels[id.item()]] for id in eval_labels]
    predictions = [[ids_to_labels[id.item()]] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [237]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.24397523701190948
Validation loss per 100 evaluation steps: 0.18071570020488298
Validation Loss: 0.1842390117005423
Validation Accuracy: 0.9563099551760467


In [238]:
from seqeval.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

ORGANISATION       0.70      0.70      0.70       274
      PERSON       0.91      0.93      0.92       368
       PLACE       0.88      0.89      0.89       637

   micro avg       0.85      0.86      0.86      1279
   macro avg       0.83      0.84      0.84      1279
weighted avg       0.85      0.86      0.86      1279



In [240]:
import os

directory = "./model/indobert-large-p2-ner"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

All files saved


In [246]:
ids_to_labels = {0: 'B-ORGANISATION',
 1: 'B-PERSON',
 2: 'B-PLACE',
 3: 'I-ORGANISATION',
 4: 'I-PERSON',
 5: 'I-PLACE',
 6: 'O'}

In [243]:
sentence = "BJ Habibie adalah Presiden Indonesia ke-3"

In [ ]:
inputs = tokenizer(sentence.split(),
                    is_split_into_words = True,
                    return_offsets_mapping=True, 
                    padding='max_length', 
                    truncation=True, 
                    max_length=MAX_LEN,
                    return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, attention_mask=mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

prediction = []
for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
  #only predictions on first word pieces are important
  if mapping[0] == 0 and mapping[1] != 0:
    prediction.append(token_pred[1])
  else:
    continue

print(sentence.split())
print(prediction)

In [ ]:
import torch
def predict(sentence, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentence.split(),
                    is_split_into_words = True,
                    return_offsets_mapping=True, 
                    padding='max_length', 
                    truncation=True, 
                    max_length=MAX_LEN,
                    return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
      #only predictions on first word pieces are important
      if mapping[0] == 0 and mapping[1] != 0:
        prediction.append(token_pred[1])
      else:
        continue

    print(sentence.split())
    print(prediction)

In [250]:
## ## https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb#scrollTo=zPDla1mmZiax

## Test

In [15]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("ageng-anugrah/indobert-large-p2-finetuned-ner")
model = AutoModelForTokenClassification.from_pretrained("ageng-anugrah/indobert-large-p2-finetuned-ner")

In [28]:
import torch
def predict(model, tokenizer, sentence):
    ids_to_labels = {
      0: 'B-ORGANISATION',
      1: 'B-PERSON',
      2: 'B-PLACE',
      3: 'I-ORGANISATION',
      4: 'I-PERSON',
      5: 'I-PLACE',
      6: 'O',
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentence.split(),
                    is_split_into_words = True,
                    return_offsets_mapping=True, 
                    return_tensors="pt")
    
    model.to(device)
    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
        #only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
        else:
            continue

    return sentence.split(), prediction

In [29]:
a, b = predict(model, tokenizer, "BJ Habibie adalah Presiden Indonesia ke-3")

## Bi LSTM

In [95]:
all_data = train_data_ner + val_data_ner + test_data_ner

In [96]:
maxlen = max([len(s) for s in all_data])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 216


In [97]:
words = list(set([word[0] for sentence in all_data for word in sentence]))
words.append("ENDPAD")

In [98]:
n_words = len(words); n_words

13088

In [99]:
tags = list(set([word[1] for sentence in all_data for word in sentence]))

In [100]:
n_tags = len(tags); n_tags

7

In [101]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [102]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = [[word2idx[w[0]] for w in s] for s in train_data_ner]

In [103]:
X_test = [[word2idx[w[0]] for w in s] for s in test_data_ner]

In [105]:
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=n_words - 1)

In [104]:
X_test = pad_sequences(maxlen=maxlen, sequences=X_test, padding="post",value=n_words - 1)

In [108]:
y = [[tag2idx[w[1]] for w in s] for s in train_data_ner]

In [109]:
y_test = [[tag2idx[w[1]] for w in s] for s in test_data_ner]

In [111]:
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag2idx["O"])

In [110]:
y_test = pad_sequences(maxlen=maxlen, sequences=y_test, padding="post", value=tag2idx["O"])

In [113]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [118]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import tensorflow as tf

In [133]:
input = Input(shape=(maxlen,))
model = Embedding(input_dim=n_words, output_dim=maxlen, input_length=maxlen)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer

In [134]:
model = Model(input, out)

In [135]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [136]:
import numpy as np

history = model.fit(X, np.array(y), batch_size=32, epochs=20, validation_split=0.05, verbose=1)

Epoch 1/20
50/50 [==============================] - 51s 952ms/step - loss: 0.3353 - precision_2: 0.9735 - recall_2: 0.8458 - val_loss: 0.1198 - val_precision_2: 0.9730 - val_recall_2: 0.9730
Epoch 2/20
50/50 [==============================] - 46s 924ms/step - loss: 0.1170 - precision_2: 0.9719 - recall_2: 0.9719 - val_loss: 0.1099 - val_precision_2: 0.9730 - val_recall_2: 0.9730
Epoch 3/20
50/50 [==============================] - 45s 902ms/step - loss: 0.1032 - precision_2: 0.9742 - recall_2: 0.9718 - val_loss: 0.0986 - val_precision_2: 0.9783 - val_recall_2: 0.9725
Epoch 4/20
50/50 [==============================] - 43s 859ms/step - loss: 0.0839 - precision_2: 0.9851 - recall_2: 0.9709 - val_loss: 0.0836 - val_precision_2: 0.9858 - val_recall_2: 0.9709
Epoch 5/20
50/50 [==============================] - 44s 888ms/step - loss: 0.0658 - precision_2: 0.9953 - recall_2: 0.9702 - val_loss: 0.0772 - val_precision_2: 0.9869 - val_recall_2: 0.9710
Epoch 6/20
50/50 [===========================

In [123]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out

def tolabel(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out

In [143]:
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = tolabel(y_test)

7/7 [==============================] - 1s 105ms/step


In [142]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [143]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 0.0%


In [131]:
pred_labels

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',

In [147]:
print(classification_report(test_labels, pred_labels, digits=4))

              precision    recall  f1-score   support

ORGANISATION     0.2718    0.2258    0.2467       124
      PERSON     0.4965    0.3382    0.4023       207
       PLACE     0.6689    0.5602    0.6098       357

   micro avg     0.5488    0.4331    0.4842       688
   macro avg     0.4791    0.3747    0.4196       688
weighted avg     0.5455    0.4331    0.4819       688



In [130]:
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

B-ORGANISATION       0.00      0.00      0.00       124
      B-PERSON       0.00      0.00      0.00       207
       B-PLACE       0.46      0.56      0.51       357
I-ORGANISATION       0.00      0.00      0.00       150
      I-PERSON       0.00      0.00      0.00       161
       I-PLACE       0.57      0.35      0.43       280
             O       0.98      1.00      0.99     43865

      accuracy                           0.98     45144
     macro avg       0.29      0.27      0.28     45144
  weighted avg       0.96      0.98      0.97     45144



## BERT Fix

In [31]:
import os
import torch
torch.cuda.set_device(5)

In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("model/indobert-large-p2-finetuned-ner")
model = AutoModelForTokenClassification.from_pretrained("model/indobert-large-p2-finetuned-ner")

In [4]:
model_name = "indobenchmark/indobert-large-p2"

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
train_sentences = [[word[0] for word in sentence] for sentence in train_data_ner]
val_sentences = [[word[0] for word in sentence] for sentence in val_data_ner]
test_sentences = [[word[0] for word in sentence] for sentence in test_data_ner]

In [8]:
train_labels = [[s[1] for s in sentence]for sentence in train_data_ner]
val_labels = [[s[1] for s in sentence]for sentence in val_data_ner]
test_labels = [[s[1] for s in sentence]for sentence in test_data_ner]

In [9]:
unique_labels = list(set([lab for label in train_labels + val_labels + test_labels for lab in label]))
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

In [10]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [11]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [12]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 20
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [13]:
class dataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.len = len(sentences)
        # self.data = dataframe
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.sentences[index]
        word_labels = self.labels[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                                  is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] != 0:
                # overwrite label
                encoded_labels[idx] = labels[i]
                i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

    def __len__(self):
        return self.len

In [14]:
training_set = dataset(train_sentences, train_labels, tokenizer, MAX_LEN)
validation_set = dataset(val_sentences, val_labels, tokenizer, MAX_LEN)
testing_set = dataset(test_sentences, test_labels, tokenizer, MAX_LEN)

In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(validation_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [90]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_labels), id2label=ids_to_labels, label2id=labels_to_ids)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [16]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [93]:
from tqdm import tqdm

def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    val_loss, val_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    nb_val_examples, nb_val_steps = 0, 0
    tr_preds, tr_labels = [], []
    val_preds, val_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader)):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = tr_loss/nb_tr_steps
        #     print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_step = tr_loss/nb_tr_steps
    
    model.eval()
    for idx, batch in enumerate(val_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        val_logits = outputs[1]
        val_loss += loss.item()

        nb_val_steps += 1
        nb_val_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = val_loss/nb_val_steps
        #     print(f"Validation loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = val_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        val_labels.extend(labels)
        val_preds.extend(predictions)

        tmp_val_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        val_accuracy += tmp_val_accuracy
    

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Validation loss epoch: {val_loss/nb_val_steps}")
    print(f"Validation accuracy epoch: {val_accuracy/nb_val_steps}")

In [94]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1


100%|█████████████████████████████████████████████████████████████████████████████████| 418/418 [05:09<00:00,  1.35it/s]


Training loss epoch: 0.299460963241346
Training accuracy epoch: 0.9067297720806089
Validation loss epoch: 0.13675836628338076
Validation accuracy epoch: 0.95662865797685
Training epoch: 2


100%|█████████████████████████████████████████████████████████████████████████████████| 418/418 [05:02<00:00,  1.38it/s]


Training loss epoch: 0.11744457139949978
Training accuracy epoch: 0.962436434063046
Validation loss epoch: 0.1350524711932214
Validation accuracy epoch: 0.956331751583912
Training epoch: 3


...

Training loss epoch: 0.0030394228440149963
Training accuracy epoch: 0.9992503036732545
Validation loss epoch: 0.2606993838949766
Validation accuracy epoch: 0.9587222208240112
Training epoch: 19


100%|█████████████████████████████████████████████████████████████████████████████████| 418/418 [05:01<00:00,  1.38it/s]


Training loss epoch: 0.003482350898820008
Training accuracy epoch: 0.9992456449721255
Validation loss epoch: 0.3173451157567368
Validation accuracy epoch: 0.9543890971572562
Training epoch: 20


100%|█████████████████████████████████████████████████████████████████████████████████| 418/418 [05:00<00:00,  1.39it/s]


Training loss epoch: 0.0028515453555604635
Training accuracy epoch: 0.9993742591529756
Validation loss epoch: 0.2889173881956354
Validation accuracy epoch: 0.9572944888995217


In [17]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            # if idx % 100==0:
            #     loss_step = eval_loss/nb_eval_steps
            #     print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.append(labels)
            eval_preds.append(predictions)
            # eval_labels.extend(labels)
            # eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            

    
    predictions = [[model.config.id2label[id.item()] for id in s] for s in eval_preds]
    labels = [[model.config.id2label[id.item()] for id in s] for s in eval_labels]
    # predictions = [[model.config.id2label[id.item()]] for id in eval_preds]
    # eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [18]:
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [19]:
labels, predictions = valid(model, testing_loader)

Validation Loss: 32.39314902576916
Validation Accuracy: 0.9537801434503727


In [41]:
from seqeval.metrics import classification_report
from  sklearn_crfsuite.metrics import flat_classification_report 

print(classification_report(labels, predictions, digits=4))
print(flat_classification_report(labels, predictions))

              precision    recall  f1-score   support

ORGANISATION     0.5630    0.6129    0.5869       124
      PERSON     0.8512    0.8841    0.8673       207
       PLACE     0.8262    0.8655    0.8454       357

   micro avg     0.7845    0.8256    0.8045       688
   macro avg     0.7468    0.7875    0.7665       688
weighted avg     0.7863    0.8256    0.8054       688

                precision    recall  f1-score   support

B-ORGANISATION       0.67      0.65      0.66       124
      B-PERSON       0.88      0.89      0.88       207
       B-PLACE       0.86      0.88      0.87       357
I-ORGANISATION       0.64      0.67      0.65       150
      I-PERSON       0.88      0.89      0.88       161
       I-PLACE       0.81      0.86      0.83       280
             O       0.99      0.98      0.98      5575

      accuracy                           0.95      6854
     macro avg       0.82      0.83      0.82      6854
  weighted avg       0.95      0.95      0.95      6854



In [97]:
from seqeval.metrics import classification_report
from  sklearn_crfsuite.metrics import flat_classification_report 

print(classification_report(labels, predictions))
print(flat_classification_report(labels, predictions))

              precision    recall  f1-score   support

ORGANISATION       0.66      0.67      0.67       274
      PERSON       0.91      0.92      0.91       368
       PLACE       0.86      0.90      0.88       637

   micro avg       0.83      0.85      0.84      1279
   macro avg       0.81      0.83      0.82      1279
weighted avg       0.83      0.85      0.84      1279

                precision    recall  f1-score   support

B-ORGANISATION       0.67      0.65      0.66       124
      B-PERSON       0.88      0.89      0.88       207
       B-PLACE       0.86      0.88      0.87       357
I-ORGANISATION       0.64      0.67      0.65       150
      I-PERSON       0.88      0.89      0.88       161
       I-PLACE       0.81      0.86      0.83       280
             O       0.99      0.98      0.98      5575

      accuracy                           0.95      6854
     macro avg       0.82      0.83      0.82      6854
  weighted avg       0.95      0.95      0.95      6854



/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [98]:
import os

directory = "./model/indobert-large-p2-finetuned-ner"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

All files saved


# NER Timex

## BERT

In [80]:
train_data_timex = open_file_pos_tag("datasets/indotimex/kompas_development_Timex.txt")
val_data_timex = open_file_pos_tag("datasets/indotimex/kompas_test_gold.txt")
test_data_timex = open_file_pos_tag("datasets/indotimex/kompas_test_gold.txt")

In [2]:
import os
import torch
torch.cuda.set_device(5)

In [81]:
model_name = "indobenchmark/indobert-large-p2"

In [82]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [83]:
train_sentences = [[word[0] for word in sentence] for sentence in train_data_timex]
val_sentences = [[word[0] for word in sentence] for sentence in val_data_timex]
test_sentences = [[word[0] for word in sentence] for sentence in test_data_timex]

In [84]:
train_labels = [[s[1] for s in sentence]for sentence in train_data_timex]
val_labels = [[s[1] for s in sentence]for sentence in val_data_timex]
test_labels = [[s[1] for s in sentence]for sentence in test_data_timex]

In [85]:
unique_labels = list(set([lab for label in train_labels + val_labels + test_labels for lab in label]))
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

In [86]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [87]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 50
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [88]:
# tokenizer = AutoTokenizer.from_pretrained("model/indobert-large-p2-finetuned-indotimex")
# model = AutoModelForTokenClassification.from_pretrained("model/indobert-large-p2-finetuned-indotimex")

In [90]:
class dataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.len = len(sentences)
        # self.data = dataframe
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.sentences[index]
        word_labels = self.labels[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                                  is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] != 0:
                # overwrite label
                encoded_labels[idx] = labels[i]
                i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

    def __len__(self):
        return self.len

In [91]:
training_set = dataset(train_sentences, train_labels, tokenizer, MAX_LEN)
validation_set = dataset(val_sentences, val_labels, tokenizer, MAX_LEN)
testing_set = dataset(test_sentences, test_labels, tokenizer, MAX_LEN)

In [92]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(validation_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [93]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [94]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_labels), id2label=ids_to_labels, label2id=labels_to_ids)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [95]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [96]:
from tqdm import tqdm

def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    val_loss, val_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    nb_val_examples, nb_val_steps = 0, 0
    tr_preds, tr_labels = [], []
    val_preds, val_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader)):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = tr_loss/nb_tr_steps
        #     print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_step = tr_loss/nb_tr_steps
    
    model.eval()
    for idx, batch in enumerate(val_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        val_logits = outputs[1]
        val_loss += loss.item()

        nb_val_steps += 1
        nb_val_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = val_loss/nb_val_steps
        #     print(f"Validation loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = val_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        val_labels.extend(labels)
        val_preds.extend(predictions)

        tmp_val_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        val_accuracy += tmp_val_accuracy
    

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Validation loss epoch: {val_loss/nb_val_steps}")
    print(f"Validation accuracy epoch: {val_accuracy/nb_val_steps}")

In [97]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1


100%|███████████████████████████████████████████████████████████████████████████████████| 13/13 [00:05<00:00,  2.60it/s]


Training loss epoch: 0.5136969249982101
Training accuracy epoch: 0.8302852607760565
Validation loss epoch: 0.3737236261367798
Validation accuracy epoch: 0.9097707672135323
Training epoch: 2


100%|███████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  2.79it/s]


Training loss epoch: 0.25033326332385725
Training accuracy epoch: 0.9317390279478569
Validation loss epoch: 0.2210555225610733
Validation accuracy epoch: 0.9418265401452418
Training epoch: 3


...

Training loss epoch: 0.003138198887777085
Training accuracy epoch: 0.998959968483223
Validation loss epoch: 0.10569087894899505
Validation accuracy epoch: 0.9775925805199372
Training epoch: 49


100%|███████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  2.77it/s]


Training loss epoch: 0.003518456039063704
Training accuracy epoch: 0.9989161698022456
Validation loss epoch: 0.09898461534508637
Validation accuracy epoch: 0.9787995984876365
Training epoch: 50


100%|███████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  2.67it/s]


Training loss epoch: 0.0037191823861436346
Training accuracy epoch: 0.9988675766414981
Validation loss epoch: 0.12696401349135808
Validation accuracy epoch: 0.9767605468742089


In [98]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            # if idx % 100==0:
            #     loss_step = eval_loss/nb_eval_steps
            #     print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [[model.config.id2label[id.item()]] for id in eval_labels]
    predictions = [[model.config.id2label[id.item()]] for id in eval_preds]
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [99]:
labels, predictions = valid(model, testing_loader)

Validation Loss: 0.11939786403225018
Validation Accuracy: 0.9773476409354245


In [124]:
from seqeval.metrics import classification_report
from sklearn_crfsuite.metrics import flat_classification_report 

print(classification_report(labels, predictions))
print(classification_report(labels, predictions, mode="strict"))
print(flat_classification_report(labels, predictions))

              precision    recall  f1-score   support

       TIMEX       0.96      0.84      0.89       805

   micro avg       0.96      0.84      0.89       805
   macro avg       0.96      0.84      0.89       805
weighted avg       0.96      0.84      0.89       805

              precision    recall  f1-score   support

       TIMEX       0.79      0.79      0.79       224

   micro avg       0.79      0.79      0.79       224
   macro avg       0.79      0.79      0.79       224
weighted avg       0.79      0.79      0.79       224

              precision    recall  f1-score   support

     B-TIMEX       0.79      0.79      0.79       224
     I-TIMEX       0.96      0.79      0.87       581
           O       0.98      1.00      0.99      8034

    accuracy                           0.98      8839
   macro avg       0.91      0.86      0.88      8839
weighted avg       0.98      0.98      0.98      8839



In [36]:
sentence = "BJ Habibie adalah Presiden Indonesia ke-3 yang lahir pada tanggal 25 Juni 1936"

In [34]:
import torch
def predict(model, tokenizer, sentence):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentence.split(),
                    is_split_into_words = True,
                    return_offsets_mapping=True, 
                    return_tensors="pt")
    
    model.to(device)
    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [model.config.id2label[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
        #only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
        else:
            continue

    return sentence.split(), prediction

In [37]:
predict(model, tokenizer, sentence)

(['BJ',
  'Habibie',
  'adalah',
  'Presiden',
  'Indonesia',
  'ke-3',
  'yang',
  'lahir',
  'pada',
  'tanggal',
  '25',
  'Juni',
  '1936'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-TIMEX', 'I-TIMEX', 'O'])

In [39]:
import os

directory = "./model/indobert-large-p2-finetuned-indotimex"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

All files saved


# Chunking

In [61]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## BERT

In [63]:
def open_file_chunk(path):
    with open(path, 'r') as f:
        train_data = []
        sentence = []
        for line in f:
            if line.strip() == "":
                if len(sentence) > 0:
                    train_data.append(sentence)
                    sentence = []
            else:
                word, _, pos = line.strip().split()
                sentence.append((word, pos))
        if len(sentence) > 0:
            train_data.append(sentence)
    return train_data

In [64]:
train_data_chunk = open_file_chunk("datasets/kethu/kethu-2.0/chunking-train.txt")
val_data_chunk = open_file_chunk("datasets/kethu/kethu-2.0/chunking-test.txt")
test_data_chunk = open_file_chunk("datasets/kethu/kethu-2.0/chunking-test.txt")

In [65]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [66]:
train_sentences = [[word[0] for word in sentence] for sentence in train_data_chunk]
val_sentences = [[word[0] for word in sentence] for sentence in val_data_chunk]
test_sentences = [[word[0] for word in sentence] for sentence in test_data_chunk]

In [67]:
train_labels = [[s[1] for s in sentence]for sentence in train_data_chunk]
val_labels = [[s[1] for s in sentence]for sentence in val_data_chunk]
test_labels = [[s[1] for s in sentence]for sentence in test_data_chunk]

In [68]:
unique_labels = list(set([lab for label in train_labels + val_labels + test_labels for lab in label]))
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

In [78]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 20
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [79]:
training_set = dataset(train_sentences, train_labels, tokenizer, MAX_LEN)
validation_set = dataset(val_sentences, val_labels, tokenizer, MAX_LEN)
testing_set = dataset(test_sentences, test_labels, tokenizer, MAX_LEN)

In [80]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(validation_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [81]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_labels), id2label=ids_to_labels, label2id=labels_to_ids)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [82]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [83]:
from tqdm import tqdm

def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    val_loss, val_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    nb_val_examples, nb_val_steps = 0, 0
    tr_preds, tr_labels = [], []
    val_preds, val_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader)):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = tr_loss/nb_tr_steps
        #     print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_step = tr_loss/nb_tr_steps
    
    model.eval()
    for idx, batch in enumerate(val_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        val_logits = outputs[1]
        val_loss += loss.item()

        nb_val_steps += 1
        nb_val_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = val_loss/nb_val_steps
        #     print(f"Validation loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = val_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        val_labels.extend(labels)
        val_preds.extend(predictions)

        tmp_val_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        val_accuracy += tmp_val_accuracy
    

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Validation loss epoch: {val_loss/nb_val_steps}")
    print(f"Validation accuracy epoch: {val_accuracy/nb_val_steps}")

In [84]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1


100%|█████████████████████████████████████████████████████████████████████████████████| 232/232 [02:47<00:00,  1.39it/s]


Training loss epoch: 0.8942780041874483
Training accuracy epoch: 0.7309766545785527
Validation loss epoch: 0.38698670433627236
Validation accuracy epoch: 0.8917795428785799
Training epoch: 2


100%|█████████████████████████████████████████████████████████████████████████████████| 232/232 [02:47<00:00,  1.39it/s]


Training loss epoch: 0.2687597471321451
Training accuracy epoch: 0.9297507406243354
Validation loss epoch: 0.25372543434302014
Validation accuracy epoch: 0.9300932135251687
Training epoch: 3


...

Training loss epoch: 0.004447908452079744
Training accuracy epoch: 0.9990459137658566
Validation loss epoch: 0.2861916886263148
Validation accuracy epoch: 0.9535616047731504
Training epoch: 19


100%|█████████████████████████████████████████████████████████████████████████████████| 232/232 [02:46<00:00,  1.39it/s]


Training loss epoch: 0.00520988745333382
Training accuracy epoch: 0.9987563710169747
Validation loss epoch: 0.3024304651318946
Validation accuracy epoch: 0.9544336535442498
Training epoch: 20


100%|█████████████████████████████████████████████████████████████████████████████████| 232/232 [02:45<00:00,  1.40it/s]


Training loss epoch: 0.00563592231365278
Training accuracy epoch: 0.9986646311258427
Validation loss epoch: 0.333159782682304
Validation accuracy epoch: 0.9480558140708901


In [85]:
labels, predictions = valid(model, testing_loader)

Validation Loss: 0.3379885861997433
Validation Accuracy: 0.9475643195167447


In [86]:
from seqeval.metrics import classification_report
from  sklearn_crfsuite.metrics import flat_classification_report 

print(classification_report(labels, predictions))
print(flat_classification_report(labels, predictions))

              precision    recall  f1-score   support

        ADJP       0.80      0.77      0.78        61
        ADVP       0.94      0.90      0.92       111
        INTJ       0.00      0.00      0.00         1
          NP       0.99      0.98      0.98      1665
          PP       0.93      0.96      0.95       216
         PRT       1.00      0.86      0.92         7
        SBAR       0.91      0.95      0.93       117
         UCP       0.50      0.75      0.60         4
          VP       0.95      0.94      0.95       370

   micro avg       0.97      0.96      0.96      2552
   macro avg       0.78      0.79      0.78      2552
weighted avg       0.97      0.96      0.96      2552

              precision    recall  f1-score   support

      B-ADJP       0.83      0.81      0.82        48
      B-ADVP       0.96      0.90      0.93       105
      B-INTJ       0.00      0.00      0.00         1
        B-NP       0.94      0.98      0.96       569
        B-PP       0.94 

/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
predict(model, tokenizer, "Saya adalah seorang pelaut")

(['Saya', 'adalah', 'seorang', 'pelaut'], ['B-NP', 'B-VP', 'B-NP', 'I-NP'])

In [89]:
import os

directory = "./model/indobert-large-p2-finetuned-chunking"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

All files saved


# Dependency Parsing

In [208]:
import nltk
import conllu
from conllu import parse_incr
from nltk.parse.transitionparser import TransitionParser
from nltk.parse.dependencygraph import DependencyGraph

In [22]:
train_file = 'datasets/UD_Indonesian-GSD/id_gsd-ud-train.conllu'
test_file = 'datasets/UD_Indonesian-GSD/id_gsd-ud-test.conllu'

import spacy
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.language import Language
from spacy.training import Example
from spacy.util import minibatch, compounding

# load data
with open(train_file, "r", encoding="utf-8") as f:
    text = f.read()

# create blank model
nlp = spacy.blank("xx")

# create custom tokenizer
def custom_tokenizer(text):
    words = []
    spaces = []
    for line in text.strip().split("\n"):
        if line.startswith("#") or line.startswith(""):
            continue
        columns = line.strip().split("\t")
        words.append(columns[1])
        spaces.append(True if columns[0].isdigit() else False)
    return Doc(Vocab(), words=words, spaces=spaces)

# set tokenizer for the pipeline
nlp.tokenizer = custom_tokenizer

# add dependency parser to the pipeline
nlp.add_pipe("parser")

# train the model
optimizer = nlp.begin_training()
for i in range(10):
    losses = {}
    batches = minibatch(text, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        examples = []
        for doc in nlp.pipe(batch):
            heads = [int(token.head.i - token.i) for token in doc]
            deps = [token.dep_ for token in doc]
            examples.append(Example.from_dict(doc, {"heads": heads, "deps": deps}))
        nlp.update(examples, sgd=optimizer, losses=losses)
    print(losses)



{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}
{'parser': 0.0}


In [23]:
with open(test_file, "r", encoding="utf-8") as f:
    test_text = f.read()

In [24]:
# evaluate the model
docs = list(nlp.pipe(custom_tokenizer(text) for text in test_text.split("\n\n") if text.strip()))
gold = []
predicted = []
for doc in docs:
    heads = [int(token.head.i - token.i) for token in doc]
    deps = [token.dep_ for token in doc]
    gold.append((heads, deps))
    predicted_heads = [int(token.head.i - token.i) for token in doc]
    predicted_deps = [token.dep_ for token in doc]
    predicted.append((predicted_heads, predicted_deps))

# print evaluation metrics
from spacy.scorer import Scorer
scorer = Scorer(nlp)
scorer.score(predicted, gold)
print(scorer.scores)

# save the model
nlp.to_disk("dependency_parser_model")

TypeError: score() takes 2 positional arguments but 3 were given

In [28]:
docs

[,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 

In [22]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
import spacy
from spacy.tokens import Doc
from spacy.vocab import Vocab

# load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('indolem/indobert-base-uncased')

# define the model
model = BertForTokenClassification.from_pretrained('indolem/indobert-base-uncased')

from torch.utils.data import Dataset

# define the dataset class
class DependencyParsingDataset(Dataset):
    def __init__(self, data):
        self.docs = [conllu_to_doc(sent) for sent in data.split('\n\n') if sent.strip() != '']
    
    def __len__(self):
        return len(self.docs)
    
    def __getitem__(self, idx):
        doc = self.docs[idx]
        input_feature = doc_to_input(doc)
        output_label = doc_to_output(doc)
        return input_feature, output_label

# define the optimizer and the loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# load the data in CoNLL-U format
with open('datasets/UD_Indonesian-PUD/id_pud-ud-test.conllu', 'r', encoding='utf-8') as f:
    data = f.read()

# define the function to convert CoNLL-U data to SpaCy Doc object
def conllu_to_doc(data):
    # create a SpaCy vocab
    vocab = Vocab()
    # create a list to store tokens
    tokens = []
    # create a list to store heads
    heads = []
    # create a list to store dependencies
    deps = []
    # loop over each line in the data
    for line in data.split('\n'):
        # check if the line is a comment or an empty line
        if not line.startswith('#') and line.strip() != '':
            # split the line into fields
            fields = line.split('\t')
            # replace the head index with 0 if it is '_'
            head = int(fields[6]) - 1 if fields[6] != '_' else 0
            # add the token to the list of tokens
            tokens.append(fields[1])
            # add the head to the list of heads
            heads.append(head)
            # add the dependency to the list of dependencies
            deps.append(fields[7])
    # create a SpaCy Doc object
    doc = Doc(vocab, words=tokens)
    # add the dependency labels to the Doc object
    for i, token in enumerate(doc):
        token.dep_ = deps[i]
    # add the head index to the Doc object
    for i, token in enumerate(doc):
        token.head = doc[heads[i]]
    # return the Doc object
    return doc

# convert the CoNLL-U data to SpaCy Doc objects
docs = [conllu_to_doc(sent) for sent in data.split('\n\n') if sent.strip() != '']

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncas

In [25]:
# define the function to convert SpaCy Doc object to input features for BERT
def doc_to_input(doc):
    # create a list to store the input tokens
    input_tokens = []
    # create a list to store the input token IDs
    input_ids = []
    # create a list to store the input attention masks
    attention_masks = []
    # loop over each token in the Doc object
    for token in doc:
        # tokenize the token using the BERT tokenizer
        token_tokens = tokenizer.tokenize(token.text)
        # add the token to the list of input tokens
        input_tokens.extend(token_tokens)
        # add the token IDs to the list of input token IDs
        token_ids = tokenizer.convert_tokens_to_ids(token_tokens)
        input_ids.extend(token_ids)
        # create the attention mask for the token
        attention_mask = [1] * len(token_tokens)
        attention_masks.extend(attention_mask)
    # create the input features
    input_features = {
        'input_ids': torch.tensor(input_ids).unsqueeze(0),
        'attention_mask': torch.tensor(attention)
    }
    return input_features

# define the function to convert SpaCy Doc object to output labels for BERT
def doc_to_output(doc):
    # create a list to store the output labels
    output_labels = []
    
    # loop over each token in the Doc object
    for token in doc:
        # add the label index to the list of output labels
        output_labels.append(token.vocab.strings[token.dep_])
        # create the output labels
        output_labels = ['O'] + output_labels + ['O']
        # create the output label IDs
        output_ids = [model.config.label2id[label] for label in output_labels]
    return output_ids

# define the training function
# define the training function
def train(model, optimizer, loss_fn, docs, batch_size=32, epochs=5):
    # set the model to training mode
    model.train()
    # loop over the epochs
    for epoch in range(epochs):
        # shuffle the data
        docs = torch.utils.data.DataLoader(docs, batch_size=batch_size, shuffle=True)
        # create a list to store the losses
        losses = []
        # loop over each batch
        for batch_docs in docs:
            # create a list to store the input features
            input_features = []
            # create a list to store the output labels
            output_labels = []
            # loop over each Doc object in the batch
            for doc in batch_docs:
                # convert the Doc object to input features
                input_feature = doc_to_input(doc)
                input_features.append(input_feature)
                # convert the Doc object to output labels
                output_label = doc_to_output(doc)
                output_labels.append(output_label)
            # pad the input features and output labels
            input_features = torch.nn.utils.rnn.pad_sequence(input_features, batch_first=True, padding_value=0)
            output_labels = torch.nn.utils.rnn.pad_sequence(output_labels, batch_first=True, padding_value=0)
            # move the input features and output labels to the device
            input_features = {key: value.to(device) for key, value in input_features.items()}
            output_labels = output_labels.to(device)
            # zero the gradients
            optimizer.zero_grad()
            # forward pass
            output = model(**input_features).logits
            # calculate the loss
            loss = loss_fn(output.view(-1, model.config.num_labels), output_labels.view(-1))
            # backward pass
            loss.backward()
            # update the parameters
            optimizer.step()
            # append the loss to the list of losses
            losses.append(loss.item())
        # print the average loss for the epoch
        print('Epoch {}/{}: loss = {:.4f}'.format(epoch+1, epochs, sum(losses)/len(losses)))

# create the dataset
dataset = DependencyParsingDataset(data)

# define the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
# move the model to the device
model.to(device)

# move the optimizer to the device
optimizer.state = {key: value.to(device) for key, value in optimizer.state.items()}

# train the model
train(model, optimizer, loss_fn, dataloader, batch_size=32, epochs=5)



TypeError: 'DataLoader' object is not subscriptable

In [27]:
docs[1]

Bagi mereka yang mengikuti transisi media sosial di Capitol Hill , ini akan sedikit berbeda . 

In [ ]:
# save the model
torch.save(model.state_dict(), 'model.pt')

In [36]:
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader

# Define the dataset class for dependency parsing
class DependencyParsingDataset(Dataset):
    def __init__(self, input_file_path, tokenizer):
        self.input_file_path = input_file_path
        self.tokenizer = tokenizer
        self.sentences = []
        self.heads = []
        self.labels = []
        self.load_data()

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx], self.heads[idx], self.labels[idx]

    def load_data(self):
        # Load the data from the input file
        with open(self.input_file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        sentence = []
        head = [-1]
        label = ["ROOT"]
        for line in lines:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if line == "":
                self.sentences.append(sentence)
                self.heads.append(head)
                self.labels.append(label)
                sentence = []
                head = [-1]
                label = ["ROOT"]
            else:
                parts = line.split("\t")
                token = parts[1]
                head_value = parts[6]
                if head_value == "_":
                    head.append(-1)
                else:
                    head.append(int(head_value))
                label.append(parts[7])
                sentence.append(token)

        if len(sentence) > 0:
            self.sentences.append(sentence)
            self.heads.append(head)
            self.labels.append(label)

    def tokenize_and_encode(self, sentence, head, label):
        tokens = []
        for i, word in enumerate(sentence):
            if i == 0:
                continue  # Skip the root token
            tokenized_word = self.tokenizer.tokenize(word)
            tokens.extend(tokenized_word)
        tokens.insert(0, "[CLS]")
        head.insert(0, -1)
        label.insert(0, "ROOT")
        tokens.append("[SEP]")
        head.append(-1)
        label.append("ROOT")
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        return input_ids, head, label

    def collate_fn(self, batch):
        sentences = [b[0] for b in batch]
        heads = [b[1] for b in batch]
        labels = [b[2] for b in batch]
        input_ids = []
        for sentence, head, label in zip(sentences, heads, labels):
            input_id, _, _ = self.tokenize_and_encode(sentence, head, label)
            input_ids.append(input_id)
        return torch.LongTensor(input_ids), torch.LongTensor(heads), labels

# Define the model configuration for dependency parsing
class DependencyParsingModel(torch.nn.Module):
    def __init__(self, bert_model, dropout=0.1):
        super().__init__()
        self.bert = bert_model
        self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids):
        outputs = self.bert(input_ids)[0]
        outputs = self.dropout(outputs)
        logits = self.linear(outputs)
        logits = logits.squeeze(-1)
        return logits

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained BERT model and tokenizer
model_name = "indolem/indobert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name).to(device)

# Define the dataset and data loader for training
train_dataset = DependencyParsingDataset("datasets/UD_Indonesian-PUD/id_pud-ud-test.conllu", tokenizer)
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=train_dataset.collate_fn)

# Define the optimizer and loss function for training
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

# Define the training loop
def train(model, optimizer, criterion, train_data_loader, device, num_epochs):
    for epoch in range(num_epochs):
        epoch_loss = 0
        num_batches = 0
        for input_ids, heads, labels in train_data_loader:
            input_ids = input_ids.to(device)
            heads = heads.to(device)
            logits = model(input_ids)
            logits = logits.view(-1, logits.size(-1))
            heads = heads.view(-1)
            loss = criterion(logits, heads)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            num_batches += 1
        epoch_loss /= num_batches
        print("Epoch {} loss: {:.4f}".format(epoch+1, epoch_loss))

# Train the model for a specified number of epochs
num_epochs = 10
model = DependencyParsingModel(bert_model).to(device)
train(model, optimizer, criterion, train_data_loader, device, num_epochs)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: The expanded size of the tensor (22088) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 22088].  Tensor sizes: [1, 512]

In [64]:
import spacy
from spacy.training import Example

# Load the CoNLL-U data
with open("datasets/UD_Indonesian-PUD/id_pud-ud-test.conllu", "r", encoding="utf-8") as f:
    data = f.read()

# Convert the data to spaCy format
nlp = spacy.blank("en")
for line in data.split("\n\n"):
    if not line.strip():
        continue
    words = []
    tags = []
    heads = []
    labels = []
    labels.append("root")
    for row in line.strip().split("\n"):
        if row.startswith("#"):
            continue
        parts = row.strip().split("\t")
        words.append(parts[1])
        tags.append(parts[3])
        if parts[6] == "_":
            heads.append(None)
        else:
            heads.append(int(parts[6]))
        labels.append(parts[7])
    doc = nlp.make_doc(" ".join(words))
    example = Example.from_dict(doc, {"heads": heads, "deps": labels})
    nlp.update([example])

# Save the trained model
nlp.to_disk("model")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [66]:
labels

['root',
 'punct',
 'mark',
 'det',
 'nsubj:pass',
 'amod',
 'nsubj:pass',
 'acl:relcl',
 'case',
 'obl',
 'flat:name',
 'advmod',
 'advmod',
 'advcl',
 'advmod',
 'punct',
 'nsubj',
 'nmod',
 'nsubj',
 'acl:relcl',
 '_',
 'advmod',
 'advmod:emph',
 'root',
 'punct',
 'punct',
 'parataxis',
 'nsubj',
 'amod',
 'nmod:poss',
 'punct',
 'appos',
 'flat:name',
 'case',
 'det',
 'obl',
 'nmod',
 'case',
 'obl:tmod',
 'nmod:tmod',
 'punct']

In [3]:
from seqeval.metrics import classification_report
from  sklearn_crfsuite.metrics import flat_classification_report 

true_labels = [["B-PER", "I-PER", "O", "O", "B-PLACE", "O", "O", "O"]]
predicted_tags = [["B-PER", "O", "O", "O", "B-PLACE", "O", "O", "B-ORG"]]
print(classification_report(true_labels, predicted_tags))
print(flat_classification_report(true_labels, predicted_tags))

              precision    recall  f1-score   support

         ORG       0.00      0.00      0.00         0
         PER       0.00      0.00      0.00         1
       PLACE       1.00      1.00      1.00         1

   micro avg       0.33      0.50      0.40         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2

              precision    recall  f1-score   support

       B-ORG       0.00      0.00      0.00         0
       B-PER       1.00      1.00      1.00         1
     B-PLACE       1.00      1.00      1.00         1
       I-PER       0.00      0.00      0.00         1
           O       0.80      0.80      0.80         5

    accuracy                           0.75         8
   macro avg       0.56      0.56      0.56         8
weighted avg       0.75      0.75      0.75         8



/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-